# Anomaly detection-risk based clustering classification
<img src = 'https://images.unsplash.com/photo-1602091118779-8974530b03ac?ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDg3fDZzTVZqVExTa2VRfHxlbnwwfHx8fA%3D%3D&ixlib=rb-1.2.1&auto=format&fit=crop&w=800&q=60' width="500" height="133">

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')
# Imports
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.family']='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False  #한글 폰트 사용시 마이너스 폰트 깨짐 해결

In [2]:
df = pd.read_pickle('../data/df_filtered.pkl')
df.head()
df.shape

,유형,용도명,제조국(원산지),과세가격($/Kg),유기식품 여부,영유아섭취대상,주문자상표부착,유전자변형식품,가온보존대상,진균수시험대상,방사선조사식품,판정결과,순중량(Kg)_log,과세가격_log
2225054,즉석조리식품,판매용,인도네시아,3.647,아니오,아니오,아니오,표시함,N,N,N,0,6.546785,7.839526
255568,배추김치,판매용,중국,0.460,아니오,아니오,아니오,해당없음,N,N,N,0,8.987322,8.210940
130945,과실주,판매용,호주,11.692,아니오,아니오,아니오,해당없음,N,N,N,0,6.916715,9.374673
2089969,기타 수산물가공품,판매용,베트남,5.400,아니오,아니오,아니오,표시함,N,N,N,0,9.287394,10.973718
193189,과실주,판매용,프랑스,2.333,아니오,아니오,아니오,해당없음,N,N,N,0,8.412055,9.259226


(68775, 14)

# 1. 데이터 전처리 
- 데이터전처리와 교차검증 분리
- SMOTE()적용을 위해 모든 특성변수는 수치화

In [3]:
data = df.copy()

In [4]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

numeric_features = data.drop(['판정결과'], axis=1).select_dtypes(include=['int64', 'float64']).columns
categorical_features = data.drop(['판정결과'], axis=1).select_dtypes(include=['object', 'category']).columns
numeric_features
categorical_features

Index(['과세가격($/Kg)', '순중량(Kg)_log', '과세가격_log'], dtype='object')

Index(['유형', '용도명', '제조국(원산지)', '유기식품 여부', '영유아섭취대상', '주문자상표부착', '유전자변형식품',
       '가온보존대상', '진균수시험대상', '방사선조사식품'],
      dtype='object')

- 범주형 변수의 target mean

In [5]:
import category_encoders as ce
encoder = ce.target_encoder.TargetEncoder(cols= categorical_features)
encoder.fit(data[categorical_features], data['판정결과'])
data[categorical_features] = encoder.transform(data[categorical_features])
data.head()

TargetEncoder(cols=Index(['유형', '용도명', '제조국(원산지)', '유기식품 여부', '영유아섭취대상', '주문자상표부착', '유전자변형식품',
       '가온보존대상', '진균수시험대상', '방사선조사식품'],
      dtype='object'))

,유형,용도명,제조국(원산지),과세가격($/Kg),유기식품 여부,영유아섭취대상,주문자상표부착,유전자변형식품,가온보존대상,진균수시험대상,방사선조사식품,판정결과,순중량(Kg)_log,과세가격_log
2225054,0.002008,0.011723,0.024262,3.647,0.011666,0.01169,0.011792,0.007928,0.011706,0.011397,0.011527,0,6.546785,7.839526
255568,0.001698,0.011723,0.018307,0.460,0.011666,0.01169,0.011792,0.012417,0.011706,0.011397,0.011527,0,8.987322,8.210940
130945,0.001619,0.011723,0.018062,11.692,0.011666,0.01169,0.011792,0.012417,0.011706,0.011397,0.011527,0,6.916715,9.374673
2089969,0.013736,0.011723,0.029974,5.400,0.011666,0.01169,0.011792,0.007928,0.011706,0.011397,0.011527,0,9.287394,10.973718
193189,0.001619,0.011723,0.004590,2.333,0.011666,0.01169,0.011792,0.012417,0.011706,0.011397,0.011527,0,8.412055,9.259226


- 실수형 변수의 표준화(dynamic time warping을 적용하기 위해 minmax로 모든 변수를 0 ~ 1로 표준화)

In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
encoder = MinMaxScaler()
encoder.fit(data[numeric_features], data['판정결과'])
data[numeric_features] = encoder.transform(data[numeric_features])
#data[numeric_features].mean(); data[numeric_features].std()
data[numeric_features].min(); data[numeric_features].max()
data.head()

MinMaxScaler()

과세가격($/Kg)     0.0
순중량(Kg)_log    0.0
과세가격_log       0.0
dtype: float64

과세가격($/Kg)     1.0
순중량(Kg)_log    1.0
과세가격_log       1.0
dtype: float64

,유형,용도명,제조국(원산지),과세가격($/Kg),유기식품 여부,영유아섭취대상,주문자상표부착,유전자변형식품,가온보존대상,진균수시험대상,방사선조사식품,판정결과,순중량(Kg)_log,과세가격_log
2225054,0.002008,0.011723,0.024262,0.000122,0.011666,0.01169,0.011792,0.007928,0.011706,0.011397,0.011527,0,0.576558,0.614300
255568,0.001698,0.011723,0.018307,0.000015,0.011666,0.01169,0.011792,0.012417,0.011706,0.011397,0.011527,0,0.811579,0.646226
130945,0.001619,0.011723,0.018062,0.000390,0.011666,0.01169,0.011792,0.012417,0.011706,0.011397,0.011527,0,0.612182,0.746260
2089969,0.013736,0.011723,0.029974,0.000180,0.011666,0.01169,0.011792,0.007928,0.011706,0.011397,0.011527,0,0.840476,0.883713
193189,0.001619,0.011723,0.004590,0.000078,0.011666,0.01169,0.011792,0.012417,0.011706,0.011397,0.011527,0,0.756182,0.736336


- Target 수치화

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
le = LabelEncoder()
label_encoder = le.fit(data['판정결과'])
data['판정결과'] = label_encoder.transform(data['판정결과'])

In [125]:
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
tf = data.sample(frac=0.2, random_state=135).copy()
df = tf.copy()
for rn, ratio in enumerate(np.arange(0.1, 0.6, 0.1)) : 
    rng = np.random.RandomState(rn)
    print(ratio)
    random_unlabeled_points = rng.rand(len(df.판정결과)) < ratio
    df.loc[random_unlabeled_points, '판정결과'] = -1
    df.판정결과.value_counts()

    X = df.drop('판정결과', axis=1)
    y = df['판정결과']
    y_predict = LabelPropagation().fit(X, y).predict(X)
    confusion_matrix(tf.판정결과.values, y_predict)
    tn, fp, fn, tp = confusion_matrix(tf.판정결과.values, y_predict).ravel()
    print(f'ratio:{ratio}인 경우, recall: {(tp/(tp+fn)*100)},  precision: {(tp/(tp+fp)*100)}')
    print('*'*100)

0.1


 0    12217
-1     1416
 1      122
Name: 판정결과, dtype: int64

array([[13623,     0],
       [  132,     0]], dtype=int64)

ratio:0.1인 경우, recall: 0.0,  precision: nan
****************************************************************************************************
0.2


 0    9777
-1    3873
 1     105
Name: 판정결과, dtype: int64

...

array([[13623,     0],
       [  132,     0]], dtype=int64)

ratio:0.4인 경우, recall: 0.0,  precision: nan
****************************************************************************************************
0.5


-1    11784
 0     1948
 1       23
Name: 판정결과, dtype: int64

array([[13623,     0],
       [  132,     0]], dtype=int64)

ratio:0.5인 경우, recall: 0.0,  precision: nan
****************************************************************************************************


In [127]:
from sklearn.semi_supervised import LabelSpreading

tf = data.sample(frac=0.2, random_state=135).copy()
df = tf.copy()

for rn, ratio in enumerate(np.arange(0.1, 0.6, 0.1)) : 
    rng = np.random.RandomState(rn)
    print(ratio)
    random_unlabeled_points = rng.rand(len(df.판정결과)) < ratio
    df.loc[random_unlabeled_points, '판정결과'] = -1
    df.판정결과.value_counts()

    X = df.drop('판정결과', axis=1)
    y = df['판정결과']
    y_predict = LabelSpreading(gamma=0.25, max_iter=20).fit(X, y).predict(X)
   
    confusion_matrix(tf.판정결과.values, y_predict)
    tn, fp, fn, tp = confusion_matrix(tf.판정결과.values, y_predict).ravel()
    print(f'ratio:{ratio}인 경우, recall: {(tp/(tp+fn)*100)},  precision: {(tp/(tp+fp)*100)}')
    print('*'*100)

0.1


 0    12217
-1     1416
 1      122
Name: 판정결과, dtype: int64

array([[13623,     0],
       [  132,     0]], dtype=int64)

ratio:0.1인 경우, recall: 0.0,  precision: nan
****************************************************************************************************
0.2


 0    9777
-1    3873
 1     105
Name: 판정결과, dtype: int64

...

array([[13623,     0],
       [  132,     0]], dtype=int64)

ratio:0.4인 경우, recall: 0.0,  precision: nan
****************************************************************************************************
0.5


-1    11784
 0     1948
 1       23
Name: 판정결과, dtype: int64

array([[13623,     0],
       [  132,     0]], dtype=int64)

ratio:0.5인 경우, recall: 0.0,  precision: nan
****************************************************************************************************


In [134]:
recalls=[]
for g in np.arange(0.1, 1, 0.05):
    tf = data.sample(frac=0.2, random_state=135).copy()
    df = tf.copy()
    for rn, ratio in enumerate(np.arange(0.1, 0.6, 0.1)) : 
        rng = np.random.RandomState(rn)
        print(ratio)
        random_unlabeled_points = rng.rand(len(df.판정결과)) < ratio
        df.loc[random_unlabeled_points, '판정결과'] = -1
        df.판정결과.value_counts()

        X = df.drop('판정결과', axis=1)
        y = df['판정결과']
        y_predict = LabelSpreading(gamma= g, max_iter=20).fit(X, y).predict(X)

        confusion_matrix(tf.판정결과.values, y_predict)
        tn, fp, fn, tp = confusion_matrix(tf.판정결과.values, y_predict).ravel()
        print(f'ratio:{ratio}인 경우, recall: {(tp/(tp+fn)*100)},  precision: {(tp/(tp+fp)*100)}')
        recalls.append(tp/(fn + tp))
        print('*'*100)

0.1


 0    12217
-1     1416
 1      122
Name: 판정결과, dtype: int64

array([[13623,     0],
       [  132,     0]], dtype=int64)

ratio:0.1인 경우, recall: 0.0,  precision: nan
****************************************************************************************************
0.2


 0    9777
-1    3873
 1     105
Name: 판정결과, dtype: int64

...

array([[13623,     0],
       [  132,     0]], dtype=int64)

ratio:0.4인 경우, recall: 0.0,  precision: nan
****************************************************************************************************
0.5


-1    11784
 0     1948
 1       23
Name: 판정결과, dtype: int64

array([[13623,     0],
       [  132,     0]], dtype=int64)

ratio:0.5인 경우, recall: 0.0,  precision: nan
****************************************************************************************************


In [137]:
np.sum(recalls)

0.0

# End